In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from qutip import *

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as offline

In [ ]:
from ipywidgets import interact

In [ ]:
import warnings
import itertools as it
import numpy as np
from numpy import pi, array, sin, cos, angle, log2, sqrt

try:
    import matplotlib.pyplot as plt
    import matplotlib as mpl
    from matplotlib import cm
    from mpl_toolkits.mplot3d import Axes3D
except:
    pass

from qutip.qobj import Qobj, isket
from qutip.states import ket2dm
from qutip.wigner import wigner
from qutip.tensor import tensor
from qutip.matplotlib_utilities import complex_phase_cmap
from qutip.superoperator import vector_to_operator
from qutip.superop_reps import to_super, _super_to_superpauli, _isqubitdims, _pauli_basis
from qutip.tensor import flatten

from qutip import settings

In [ ]:
offline.init_notebook_mode()

## Parameters

In [ ]:
N = 20

In [ ]:
def plot_wigner2(rho, fig=None, ax=None, figsize=(6, 6),
                cmap='Rdbu', alpha_max=7.5, colorbar=True,
                method='clenshaw', projection='2d'):
    """
    Plot the the Wigner function for a density matrix (or ket) that describes
    an oscillator mode.
    Parameters
    ----------
    rho : :class:`qutip.qobj.Qobj`
        The density matrix (or ket) of the state to visualize.
    fig : a matplotlib Figure instance
        The Figure canvas in which the plot will be drawn.
    ax : a matplotlib axes instance
        The axes context in which the plot will be drawn.
    figsize : (width, height)
        The size of the matplotlib figure (in inches) if it is to be created
        (that is, if no 'fig' and 'ax' arguments are passed).
    cmap : a matplotlib cmap instance
        The colormap.
    alpha_max : float
        The span of the x and y coordinates (both [-alpha_max, alpha_max]).
    colorbar : bool
        Whether (True) or not (False) a colorbar should be attached to the
        Wigner function graph.
    method : string {'clenshaw', 'iterative', 'laguerre', 'fft'}
        The method used for calculating the wigner function. See the
        documentation for qutip.wigner for details.
    projection: string {'2d', '3d'}
        Specify whether the Wigner function is to be plotted as a
        contour graph ('2d') or surface plot ('3d').
    Returns
    -------
    fig, ax : tuple
        A tuple of the matplotlib figure and axes instances used to produce
        the figure.
    """

    if isket(rho):
        rho = ket2dm(rho)

    xvec = np.linspace(-alpha_max, alpha_max, 200)
    W0 = wigner(rho, xvec, xvec, method=method)
    W, yvec = W0 if type(W0) is tuple else (W0, xvec)
    wlim = abs(W).max()
    
    if projection == '2d': 
        df = go.Heatmap(
                            z=W, x=xvec, y=yvec, connectgaps=True, zsmooth='best', 
                            colorscale=cmap, zmid=0, showscale=colorbar)
        return df

    elif projection == '3d':
        plofig = dict(type='surface', x=xvec, y=yvec, z=W0, colorscale=cmap, 
                   cmin=-wlim, cmax=wlim, showscale=colorbar)
        return plofig

    else:
        raise ValueError('Unexpected value of projection keyword argument.')

In [ ]:
def plot_wigner_2d_3d2(psi):
    plofig = make_subplots(rows=1, cols=2, specs=[[{'is_3d': False}, {'is_3d': True}]])
    fig2d = plot_wigner2(psi,  alpha_max=6)
    fig3d = plot_wigner2(psi,  alpha_max=6, projection='3d')
    plofig.append_trace(fig2d, 1, 1)
    plofig.append_trace(fig3d, 1, 2)
    offline.iplot(plofig)

## Vacuum state: $\left|0\right>$

In [ ]:
psi = basis(N, 0)
plot_wigner_2d_3d2(psi)

## Thermal states

In [ ]:
psi = thermal_dm(N, 2)
plot_wigner_2d_3d2(psi)

## Coherent states: $\left|\alpha\right>$

In [ ]:
psi = coherent(N, 2.0)
plot_wigner_2d_3d2(psi)

In [ ]:
psi = coherent(N, -1.0)
plot_wigner_2d_3d2(psi)

## superposition of coherent states

In [ ]:
psi = (coherent(N, -2.0) + coherent(N, 2.0)) / np.sqrt(2)
plot_wigner_2d_3d2(psi)

In [ ]:
psi = (coherent(N, -2.0) - coherent(N, 2.0)) / np.sqrt(2)
plot_wigner_2d_3d2(psi)

In [ ]:
psi = (coherent(N, -2.0) + coherent(N, -2j) + coherent(N, 2j) + coherent(N, 2.0)).unit()
plot_wigner_2d_3d2(psi)

In [ ]:
psi = (coherent(N, -2.0) + coherent(N, -1j) + coherent(N, 1j) + coherent(N, 2.0)).unit()
plot_wigner_2d_3d2(psi)

In [ ]:
# NN = 8
# 
# for n in range(NN):
#     psi = sum([coherent(N, 2*np.exp(2j * np.pi * m / (n + 2))) for m in range(n + 2)]).unit()
#     plot_wigner_2d_3d2(psi)

In [ ]:
NN = 8

def intecoh(i):
    psi = sum([coherent(N, 2*np.exp(2j * np.pi * m / (n + 2))) for m in range(n + 2)]).unit()
    plot_wigner_2d_3d2(psi)
    
interact(intecoh, i=list(range(NN)))

### mixture of coherent states

In [ ]:
psi = (coherent_dm(N, -2.0) + coherent_dm(N, 2.0)) / np.sqrt(2)
plot_wigner_2d_3d2(psi)

## Fock states: $\left|n\right>$

In [ ]:
from IPython.display import display

In [ ]:
# for n in range(6):
#     psi = basis(N, n)
#     plot_wigner_2d_3d2(psi)

In [ ]:
def drgr(i):
        plot_wigner_2d_3d2(basis(N, i))
interact(drgr, i=list(range(10)))

## Superposition of Fock states

In [ ]:
# NN = MM = 5
# 
# plofig = make_subplots(rows=NN, cols=MM)
# 
# for n in range(NN):
#     for m in range(MM):
#         psi = (fock(N, n) + fock(N, m)).unit()
#         plofig.append_trace(plot_wigner2(psi, colorbar=False), n+1, m+1)
#         
# offline.iplot(plofig)

## Squeezed vacuum states

In [ ]:
psi = squeeze(N, 0.5) * basis(N, 0)
display(plot_wigner_2d_3d2(psi))

psi = squeeze(N, 0.75j) * basis(N, 0)
display(plot_wigner_2d_3d2(psi))

psi = squeeze(N, -1) * basis(N, 0)
display(plot_wigner_2d_3d2(psi))

### Superposition of squeezed vacuum

In [ ]:
psi = (squeeze(N, 0.75j) * basis(N, 0) - squeeze(N, -0.75j) * basis(N, 0)).unit()
display(plot_wigner_2d_3d2(psi))

In [ ]:
### Mixture of squeezed vacuum

In [ ]:
psi = (ket2dm(squeeze(N, 0.75j) * basis(N, 0)) + ket2dm(squeeze(N, -0.75j) * basis(N, 0))).unit()
display(plot_wigner_2d_3d2(psi))

## Displaced squeezed vacuum

In [ ]:
psi = displace(N, 2) * squeeze(N, 0.75) * basis(N, 0)
display(plot_wigner_2d_3d2(psi))

### Superposition of two displaced squeezed states

In [ ]:
psi = (displace(N, -1) * squeeze(N, 0.75) * basis(N, 0) - displace(N, 1) * squeeze(N, -0.75) * basis(N, 0)).unit()
display(plot_wigner_2d_3d2(psi))

In [ ]:
from qutip.ipynbtools import version_table

version_table()